In [1]:
import os
import time
import errno
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import base64
import pickle

from numpy import mean
from log_parser import log_parser
from mpl_toolkits.mplot3d import Axes3D

try:
    from StringIO import StringIO
except ModuleNotFoundError:
    # Python 3.
    from io import StringIO

# Settings of Jupyter notebook HTML

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width: 100% !important; }</style>"))
display(HTML("<style>.container { font-size: 9.5px; !important; }</style>"))
display(HTML("<style>.rendered_html { font-size: 15px; color: red; }</style>"))
display(HTML("<style>.rendered_markdown { color: red; }</style>"))

# Initialization
### Pathes

In [3]:
default_file_name = 'dumpState.log'
file_name = 'G981V_TIE_3GMEM_30CPU_dumpstate_bottleneck_hint.txt'

#lp = log_parser('/home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/dumpState_1593794288665.log')
lp = log_parser('/home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/{}'.format(default_file_name))
ssrm = log_parser('/home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/SSRM.log')

Reading /home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/dumpState.log file is completed with 839224 lines

Reading /home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/SSRM.log file is completed with 101 lines



### Values

In [4]:
pData1 = [
    'MEMORY INFO', 
    'MEMSIZE INFO',
    'MEMLOGGER DUMPSTATE',
    'SERVICE CRITICAL cpuinfo',
    'SERVICE CRITICAL SurfaceFlinger',
    'compressed core&heap dump',
    'SYSTEM LOG',
    'EVENT LOG',
    'KERNEL LOG',
    'SEC LOG',
    'LAST KMSG',
    'AP RESET INFO',
    'FILESYSTEMS & FREE SPACE',
    'SERVICE content',
    'SERVICE dropbox',
    'SERVICE accessibility',
    'SERVICE batterystats',
    'SERVICE HIGH meminfo',
    'SERVICE package',
    'SERVICE wifi',
    'SERVICE procstats',
    'OLOG',
    'ACTIVITY MANAGER MARs',
    'ACTIVITY MANAGER PENDING INTENTS',
    'ACTIVITY MANAGER BROADCAST STATE',
    'ACTIVITY MANAGER LMK KILLS',
    'ACTIVITY MANAGER RUNNING PROCESSES',
    'ACTIVITY MANAGER MARs',
    'STORAGE BUFFER',
    'SYSTEM PROPERTIES',
    'CPU INFO',
    'VM TRACES JUST NOW',
    'MMC blk0 error',
    'MMC blk1 error',
    'DUMP OF SERVICE binder_calls_stats',
    'DUMP OF SERVICE connmetrics',
    'DUMP OF SERVICE gfxinfo',
    'DUMP OF SERVICE display',
    'DUMP OF SERVICE telephony.registry',
    'DUMP OF SERVICE wifiscanner',
    'DUMP OF SERVICE binder_calls_stats',
    'DUMP OF SERVICE settings',
    'DUMP OF SERVICE scontext',
    'DUMP OF SERVICE sdhms',
    'DUMP OF SERVICE input_method',
    'DUMP OF SERVICE CRITICAL SurfaceFlinger',
    'DUMP OF SERVICE desktopmode',
    'DUMP OF SERVICE fingerprint',
    'DUMP OF SERVICE gamemanager',
    'DUMP OF SERVICE package',
    'LOG STATISTICS',
    'TSP RAW DATA',
    'beginning of main',
    'beginning of radio',
    'STORAGED IO INFO',
    'PowerAnomaly Battery Dump',
    'UFS ERR SUM',
    'UFS FATAL CNT',
    'UFS OP CNT',
    'UFS QUERY CNT',
    'UFS UIC CMD CNT',
    'UFS UIC ERR CNT',
    'NETWORK DEV INFO',
    'PROCESSES AND THREADS',
    'RADIO LOG',
    'APP SERVICES PLATFORM',
    'APP SERVICES NON-PLATFORM',
    'APP PROVIDERS NON-PLATFORM',
]

lp.run(arg1=pData1)

#display(lp.log_sections['DUMP OF SERVICE binder_calls_stats'])

### Functions

In [5]:
def grep(lines, searchers, n=99999, show=True):
    result = list()
    number_of_lines = 0
    number_of_negative_lines = 0
    negative_lines = list()

    if n < 0:
        number_of_negative_lines = (-1) * n
        
    for line in lines:
        if len(line) > 0:
            for searcher in searchers:
                if re.search(searcher, line):
                    if n >= 0:
                        number_of_lines = n
                    else:
                        for negative_line in negative_lines:
                            if show:
                                print(negative_line.strip())
                            result.append(negative_line.strip())
                        if show:
                            print(line)
                        result.append(line.strip())
                        
            if number_of_lines > 0:
                if show:
                    print(line.strip())
                result.append(line.strip())
                number_of_lines -= 1
                #if number_of_lines == 0:
                    #print("\n")
                    #result.append('')

            if number_of_negative_lines > 0:
                negative_lines.append(line)
                number_of_negative_lines -= 1
            elif n < 0:
                negative_lines.pop(0)
                negative_lines.append(line)
                
    if len(result) == 0:
        print("--- Nothing ---")
        
    return result

In [6]:
def pickedup(src, searcher, index=0):
    separators = ['/', ',', ' ']
    
    if searcher not in src:
        return None
    
    splited = src.split(searcher)
    if splited <= 0:
        return None
    
    numbers = re.findall('-?\d+', splited[1])
    if len(numbers) > index:
        return int(numbers[index])
    
    return None

In [7]:
def DictFromPickedUp(src, searchers):
    result = dict()
    for key in searchers.keys():
        result[key] = list()

    tmp = dict()
    for line in src:
        flag = False
        for key in searchers.keys():
            tmp[key] = pickedup(line, key, searchers[key])
            if tmp[key] is None:
                flag = True

        if flag == True:
            continue

        for key in searchers.keys():
            result[key].append(tmp[key])
            
    return result

In [8]:
def orderbyeq(src, index=0):
    result = dict()
    items = re.split('[ ]', src)
    for item in items:
        if '=' in item and item[0] == '[':
            item = item[1:]
            splited = item.split('=')
            key = splited[0]
            numbers = re.findall('\d+', splited[1])
            if numbers > 0:
                result[key] = int(numbers[0])
                
    return result

In [9]:
def DataframeFromDictSortByCount(rawdata):
    df = pd.DataFrame.from_dict(rawdata, orient='index')
    df['count'] = df.count(axis=1, numeric_only=True)
    df = df.sort_index(by='count', axis=0, ascending=False)
    df = df.drop(columns=['count'])    
    return df

In [10]:
def PlotFromDict(rawdata, threshold, title='', ylabel=''):
    df = pd.DataFrame.from_dict(rawdata, orient='index')
    for key in df.index:
        if max(rawdata[key]) > threshold:
            s = pd.Series(rawdata[key])
            ax = s.plot(figsize=(30,5), label=key, legend=True)
            ax.axhline(y=threshold, xmin=-1, xmax=1, color='r', linestyle='--', lw=2)
            ax.set_title(title)
            ax.set_ylabel(ylabel)

In [11]:
def MarkupLCDON(lines):
    LCD_ON = list()
    marker = ''
    for line in lines:
        if re.search('LCD', line):
            if re.search(' ON ', line):
                LCD_ON.append(line)
                marker = ' LCDON >>> '
                print(line)
            elif re.search(' OFF ', line):
                LCD_ON.append(line)
                marker = ''
                print(line)
        else:
            LCD_ON.append(marker + line)
            print(marker + line)
    return LCD_ON

In [12]:
def str2num(string):
    try:
        number = int(string)
    except:
        number = -1
        
    return number

def GetTime(line, type=None):
    Days = [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    splited_items = line.split(' ')
    Date = 0
    for item in splited_items:
        if Date == 0:
            if len(item) > 4 and item[-3] is '-':
                Month = str2num(item[-5:-3])
                Day = str2num(item[-2:])
                if Month >= 0 and Day >= 0 :
                    Date = (Days[Month] + Day) * 24 * 60 * 60 * 1000
        else:
            if type is 'SSRM':
                item += '.000'

            if len(item) > 11 and item[2] is ':' and item[5] is ':' and item[8] is '.':
                Hour = str2num(item[0:2])
                Minute = str2num(item[3:5])
                Second = str2num(item[6:8])
                MiliSecond = str2num(item[9:12])
                if Hour >= 0 and Minute >= 0 and Second >= 0 and MiliSecond >= 0:
                    return Date + ((Hour * 60 + Minute) * 60 + Second) * 1000 + MiliSecond
            Date = 0
            
    return -1

def GetTime2(line, type=None):
    Days = [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    splited_items = line.split(' ')
    Date = 0
    for item in splited_items:
        if Date == 0:
            if len(item) > 4 and item[-3] is '-':
                Month = str2num(item[-5:-3])
                Day = str2num(item[-2:])
                if Month >= 0 and Day >= 0 :
                    TotalDays = 0
                    for i in range(1, Month):
                        TotalDays += Days[i]
                    Date = (TotalDays + Day) * 24 * 60 * 60 * 1000
        else:
            if type is 'SSRM':
                item += '.000'

            if len(item) > 11 and item[2] is ':' and item[5] is ':' and item[8] is '.':
                Hour = str2num(item[0:2])
                Minute = str2num(item[3:5])
                Second = str2num(item[6:8])
                MiliSecond = str2num(item[9:12])
                if Hour >= 0 and Minute >= 0 and Second >= 0 and MiliSecond >= 0:
                    return Date + ((Hour * 60 + Minute) * 60 + Second) * 1000 + MiliSecond
            Date = 0
            
    return -1

def GetKernelTime(line):
    splited = line.split('[')
    splited = splited[1].split(']')
    splited = splited[0].split('.')

    second = str2num(splited[0])
    if second < 0:
        return -1
    
    nanosecond = str2num(splited[1])
    if nanosecond < 0:
        return -1
    
    return (int)(second * 1000 + (int)(nanosecond / 1000))

In [13]:
def GetSsrmDump(lines):
    CompressedEncriptedStr = ""
    flag = False

    for line in lines:
        if 'SSRM MEMORY DUMP' in line and flag is True:
            Trimed = line[:-1]
            CompressedEncriptedStr += Trimed
            break

        if flag is True :
            Trimed = line[:-1]
            #zlib.decompress(Trimed, 16+zlib.MAX_WBITS)
            #Unziped = gz.decompress(Trimed).decode()
            #Unziped = gzip.GzipFile(fileobj=Trimed)
            #CompressedEncriptedStr.append(Unziped)
            #print(Unziped)
            CompressedEncriptedStr += Trimed

        if 'SSRM MEMORY DUMP' in line and flag is False:
            flag = True

    if "SSRM MEMORY DUMP" in CompressedEncriptedStr:
        CompressedEncriptedStr = CompressedEncriptedStr.split("SSRM MEMORY DUMP")[0]

    return CompressedEncriptedStr

In [14]:
def DecodeSsrm(CompressedEncriptedStr):
    Decoded = base64.b64decode(CompressedEncriptedStr)       

    sfp = StringIO()
    sfp.write(Decoded)
    sfp.seek(0)
    gfp = gzip.GzipFile(fileobj=sfp, mode='rb')
    uncompressed = gfp.read()
    sfp.close()
    gfp.close()

    try:
        del ssrm_list[:]
    except:
        print("---")
    ssrm_list = uncompressed.split('\n')
    
    return ssrm_list

In [15]:
lines = lp.log_sections['DUMP OF SERVICE sdhms']
#file_lines
CompressedEncriptedStr = GetSsrmDump(lines)

if len(CompressedEncriptedStr) > 0:
    ssrm.file_lines = DecodeSsrm(CompressedEncriptedStr)
    display(ssrm.file_lines)
else:
    display("SSRM file was loaded.")

'SSRM file was loaded.'

In [16]:
if len(CompressedEncriptedStr) is 0:
    CompressedEncriptedStr = GetSsrmDump(lp.file_lines)
    
    if len(CompressedEncriptedStr) > 0:
        ssrm.file_lines = DecodeSsrm(CompressedEncriptedStr)
        #display(ssrm.file_lines)
        display("SSRM MEMORY data was parsed.")
    else:
        display("SSRM file was loaded.")

---


'SSRM MEMORY data was parsed.'

In [17]:
def ConvertEsternArabicToWestern(EsternStr) :
    buckArab = {"'":"ء", "|":"آ", "?":"أ", "&":"ؤ", "<":"إ", "}":"ئ", "A":"ا", "b":"ب", "p":"ة", "t":"ت", "v":"ث", "g":"ج", "H":"ح", "x":"خ", "d":"د", "*":"ذ", "r":"ر", "z":"ز", "s":"س", "$":"ش", "S":"ص", "D":"ض", "T":"ط", "Z":"ظ", "E":"ع", "G":"غ", "_":"ـ", "f":"ف", "q":"ق", "k":"ك", "l":"ل", "m":"م", "n":"ن", "h":"ه", "w":"و", "Y":"ى", "y":"ي", "F":"ً", "N":"ٌ", "K":"ٍ", "~":"ّ", "o":"ْ", "u":"ُ", "a":"َ", "i":"ِ", "0":"٠", "1":"١", "2":"٢", "3":"٣", "4":"٤", "5":"٥", "6":"٦", "7":"٧", "8":"٨", "9":"٩"}
    WesternStr  = list()
    for line in EsternStr:
        for k, v in buckArab.iteritems():
            #print("{}  -  {}".format(k, v))
            line = line.replace(v, k)  
                
        WesternStr.append(line)            

    return WesternStr

ssrm.file_lines = ConvertEsternArabicToWestern(ssrm.file_lines)

In [18]:
if len(ssrm.file_lines):
    ssrm_head = list()
    ssrm_contextA = list()
    ssrm_contextB = list()
    ssrm_tail = list()
    ssrm_step = 0
    ssrm_time = 0
    ssrm_previous_time = 0

    for line in ssrm.file_lines:
        ssrm_time = GetTime2(line, 'SSRM')
        if ssrm_step is 0:
            if (ssrm_time < 0):
                ssrm_head.append(line)
            else:
                ssrm_step += 1

        if ssrm_step is 1:
            if (ssrm_previous_time - 10000) <= ssrm_time:
                ssrm_contextA.append(line)
                #print("STEP1  {} {} - {}".format(ssrm_previous_time, ssrm_time, line))
            else:
                ssrm_step += 1

        if ssrm_step is 2:
            if (ssrm_time > 0):
                ssrm_contextB.append(line)
                #print("STEP2  {} {} - {}".format(ssrm_previous_time, ssrm_time, line))
            else:
                ssrm_step += 1            

        if ssrm_step is 3:
            ssrm_tail.append(line)
            
        ssrm_previous_time = ssrm_time

    ssrm.file_lines = ssrm_head + ssrm_contextB + ssrm_contextA + ssrm_tail
    
    file = open('/home/ubuntu/Android/lisa_old/ipynb/performance/testingdata/SSRM_.log', 'w+')
    for line in ssrm.file_lines:
        file.writelines(line + '\n')
        print(line)
    file.close()
    #display(ssrm.file_lines)

SSRM SETTINGS ENGINE
TOP DUMP Based on SIOP Level============================================================================================================================================================================================================
[COM] [(SIOP Level)(AP_temp,PST_temp,BAT_temp,USB_temp,CHG_temp,PA_temp,CP_temp,MMW_temp,WIFI_temp,CF_temp,BLK_temp,SUBBAT_temp,LRP_temp,LRP2_temp<LRP2_type>,(BAT_level,Inow_avg,Isys_avg)(RSSI[common,Gsm/Cdma/Lte],Radio_data,Radio_RxTput,Radio_TxTput,Wifi_data,Wifi_signal_Level,Wifi_RxTput,Wif_TxTput)(Brightness,FPS,HRR),(Vidoe_Audio,AudioDevice:Volume)]
[SET] [NDX0XWXOXXXXXXX] => [PowerSavingMode/UltraPowerSavingMode/EmgergencyMode/NormalMode, ChargingStatus(aFc, Ac, Usb, Wireless, Charge, Discharge), Cooldown, KnoxMode, Screen, AOD, AutoBright, DisplayType, HrrMode, CameraOn, MusicOn, VideoPlaying, UsbTethering, CallState, SilentLog, Radio or WiFi, BTEnable, BTConnect / WiFi Info / BT Info]
[CF:CPUFreq, GF:GPUFreq, CC:CpuCoreNum, CG:C

In [19]:
lines = lp.log_sections['SERVICE content']
tmp = grep(lines,['Uptime'], 1)

Uptime: 18d11h05m36s


* Description

The attached log was gotten right after rebooting. <br>
So, there was no useful information for analyzing the issues you were faced with.<br>
Please report the issue right after it happens without rebooting and elaborate on it. <br>
For better analysis, we need the name of application that is related to the issue and the time when it happens. <br>


로그 확인시 재부팅 직후 확보된 로그로 문제 상황을 포함하지 않은 것으로 보입니다.<br>
[문제 발생시 삼성 멤버스를 통하여 덤프와 함께 문제점 등록을 권장합니다.]<br>
삼성멤버스를 통하여 수집되는 로그는 크기가 한정되어 있어 문제 상황을 포함하지 않을 수 있으니,<br>
문제 재현시 재부팅 없이 바로 오류 보고 부탁드리며, 문제 발생 앱, 문제 발생 시각 등을 정확히 명시해 주시기 바랍니다.<br>
느림 또는 멈춤에 대해서도 보다 상세한 설명 부탁드립니다.<br>
예를 들어, 앱 진입 속도가 느림, 진입시 애니메이션이 부드럽지 않음, 앱 내부에서 스크롤이 부드럽지 않음, 게임 로딩이 느림, 게임 플레이시 부드럽지 않음, 인터넷 사용 앱에서 사진 등의 로딩이 느림 등등 구체적으로 설명해 주시면 됩니다.<br>

# <span style="color:blue">Base</span> 

2022-02-10 11:20:14

N986BXXS3EULH:user

In [20]:
file = open('problem_history.txt')

problem_history = dict()
for line in file:
    if ':' in line:
        splited = line.split(':')
        if len(splited) > 0:
            if splited[0] not in problem_history:
                problem_history[splited[0]] = list()
            problem_history[splited[0]].append(splited[1])
    #print(line)
file.close()

for history in sorted(problem_history.keys()):
    for line in problem_history[history]:
        print(history + ' : ' + line[:-1])
#display(problem_history)


F916NKSU1EULF : Unfolding시 display swap 동작이 끝나기 전에, config change 가 발생할 경우, display config 와 globla config 의 diff 가 남게 되어, waitForConfig 상태로 freeze 가 풀리지 않는 현상[P220221-03031]
F916NKSU1EVAA : Unfolding시 display swap 동작이 끝나기 전에, config change 가 발생할 경우, display config 와 globla config 의 diff 가 남게 되어, waitForConfig 상태로 freeze 가 풀리지 않는 현상[P220221-03031]
F926 : TE 신호 비정상으로 인해 20fps로 동작하는 display h/w문제 [P220224-06090, P220315-02136]
G990EXXS1BVB8 : REFRESH_RATE_MODE_NORMAL - 간헐적으로 30fps 동작하는 문제, aod 진입, REFRESH_RATE_MODE_SEAMLESS - 지문 해제후 lcd on 시 확인
G991BXXS4BVAA : REFRESH_RATE_MODE_NORMAL - 간헐적으로 30fps 동작하는 문제, aod 진입, REFRESH_RATE_MODE_SEAMLESS - 지문 해제후 lcd on 시 확인
G991BXXU4BULF : REFRESH_RATE_MODE_NORMAL - 간헐적으로 30fps 동작하는 문제, aod 진입, REFRESH_RATE_MODE_SEAMLESS - 지문 해제후 lcd on 시 확인
G991BXXU4BVB1 : REFRESH_RATE_MODE_NORMAL - 간헐적으로 30fps 동작하는 문제, aod 진입, REFRESH_RATE_MODE_SEAMLESS - 지문 해제후 lcd on 시 확인
G998BXXU4BULF : REFRESH_RATE_MODE_NORMAL - 간헐적으로 30fps 동작하는 문제, aod 진입, REFRESH_RATE_MODE_S

In [21]:
splited = lp.file_lines[1].split(' ')
dumptime = splited[2] + ' ' + splited[3][:-1]
splited = lp.file_lines[5].split('/')
version = splited[4]

print(version)
print(dumptime)

with open('version_history.pkl', 'rb') as f:
    version_history = pickle.load(f)

#version_history = dict()
if version not in version_history:
    version_history[version] = list()
if dumptime not in version_history[version]:
    version_history[version].append(dumptime)

with open('version_history.pkl', 'wb') as f:
    pickle.dump(version_history, f)

T515XXU8CVB1:user
2022-04-26 19:53:29


In [22]:
for cur_version in sorted(version_history.keys()):
    if version == cur_version:
        print('')
        print('>>> {} - {}'.format(cur_version, version_history[cur_version]))
        for problem_version in problem_history:
            if problem_version in version:
                for line in problem_history[problem_version]:
                    print('     - ' + line)
        print('')
    else:
        print('{} - {}'.format(cur_version, version_history[cur_version]))

A015MUBS4BUL1:user - ['2022-04-08 10:44:58']
A022MUBS3BVB2:user - ['2022-04-24 20:45:34']
A037MUBU2AVA6:user - ['2022-03-13 13:31:40']
A105MUBS7CUK1:user - ['2022-04-15 10:32:44', '2022-04-16 20:14:38']
A107FXXU8CUL2:user - ['2022-03-15 09:19:55']
A107MUBU5CUH1:user - ['2022-04-18 16:40:32']
A107MUBU6CVB1:user - ['2022-04-14 20:34:17']
A115MUBU2BUK4:user - ['2022-02-09 21:18:45']
A125FXXS2BVC1:user - ['2022-04-18 18:28:56']
A125FXXU2BVB3:user - ['2022-04-16 03:40:18']
A125MUBU2BVB3:user - ['2022-04-14 15:13:27']
A125USQU3BVB1:user - ['2022-04-21 21:39:58', '2022-04-24 11:03:37']
A127MUBS4AVA3:user - ['2022-04-20 13:13:54']
A127MUBU3AUJ5:user - ['2022-03-20 20:59:44']
A207MUBU3CUL1:user - ['2022-04-01 11:41:30', '2022-04-16 18:30:58']
A217FXXS7CUL2:user - ['2022-02-22 13:23:33']
A225FXXU3AVB1:user - ['2022-04-18 05:13:00']
A305GUBU8CUJ1:user - ['2022-04-15 19:48:56']
A315FXXU1CVA1:user - ['2022-02-27 06:11:33']
A315GDXS1CVB2:user - ['2022-04-14 18:06:58']
A320FLXXS9CTK3:user - ['2022-03

In [23]:
lines = lp.log_sections['DUMP OF SERVICE display']
tmp = grep(lines,['mDefaultViewport'], 1)

--- Nothing ---


###### bugreport_mode

 * BUGREPORT_DEFAULT <br>
   : adb shell bugreport
   
 * BUGREPORT_FULL<br>
   : Settings > bug report

 * BUGREPORT_INTERACTIVE<br>
   : Settings > bug report<br>
   : Settings > Bug report shortcut 

* 16xxxxxx ns : 60 hz 
* 8xxxxxx ns : 120 hz

In [24]:
lines = lp.log_sections['compressed core&heap dump']
tmp = grep(lines,['VSYNC period'], 1)
lines = lp.log_sections['SERVICE CRITICAL SurfaceFlinger']
tmp = grep(lines,['VSYNC period'], 1)

--- Nothing ---
--- Nothing ---


In [25]:
lines = lp.log_sections['SYSTEM LOG']
tmp = grep(lines,['dumpstate: begin', 'dumpstate: done', 'dumpstate info: id', 'dumpstate: do_zip_file'], 1)

04-26 19:53:29.968  root 19008 19008 I dumpstate: do_zip_file: 0 do_vibrate: 0 use_socket: 1 use_control_socket: 0 do_screenshot: 0 is_remote_mode: 0 show_header_only: 0 do_start_service: 0 telephony_only: 0 wifi_only: 0 do_progress_updates: 0 fd: -1 bugreport_mode: BUGREPORT_DEFAULT dumpstate_hal_mode: DEFAULT limited_only: 0 args: /system/bin/dumpstate -s
04-26 19:53:29.973  root 19008 19008 I dumpstate: dumpstate info: id=1, args='/system/bin/dumpstate -s', bugreport_mode= BUGREPORT_DEFAULT bugreport format version: 2.0


In [26]:
lines = lp.log_sections['SYSTEM LOG']
tmp = grep(lines,['setForcedDisplaySizeDensity'], 1)

--- Nothing ---


* the number of installed packages

In [27]:
lines = lp.log_sections['ACTIVITY MANAGER MARs']
tmp = grep(lines,['mManagedPackage'], 1)

--- Nothing ---


* debug level<br>
    high : 0x4948<br>
    mid : 0x494d<br>
    low : 0x4f4c

In [28]:
lines = lp.log_sections['SYSTEM PROPERTIES']
tmp = grep(lines,['debug_level'], 1)

--- Nothing ---


* Calculation for gap between android time and kernel time.

In [29]:
lines = lp.log_sections['KERNEL LOG']
sync_logs = grep(lines,['@Sync'], 1, show=True)

gapoftime = list()
for sync_log in sync_logs:
    androidtime = GetTime(sync_log)
    kerneltime = GetKernelTime(sync_log)

    if androidtime < 0 or kerneltime < 0:
        continue

    print("{} - {} = {}").format(androidtime, kerneltime, androidtime - kerneltime)
    gapoftime.append(androidtime - kerneltime)

GAP_OF_TIME = 0

if len(gapoftime) > 0:
    GAP_OF_TIME = int(mean(gapoftime) + 0.5)
    #display(gapoftime)
    display("Average : {}".format(GAP_OF_TIME))
    GAP_OF_TIME = gapoftime[0]
    display("Current : {}".format(GAP_OF_TIME))   

--- Nothing ---


# <span style="color:blue">CPU</span> 

The sampling time should be checked.
If the term is short, the cpu usage infor is not important.

* CPU frequency table and max frequency

In [30]:
lines = lp.log_sections['SERVICE batterystats']
CPU_freqs = grep(lines,['CPU freqs'], 1)
CPU_MAX_FREQ = 2000000

if len(CPU_freqs) > 0:
    CPU_MAX_FREQ = 0
    CPU_FREQ_TABLE = CPU_freqs[0].split(':')[1]
    for CPU_freq in CPU_FREQ_TABLE.split(' '):
        if len(CPU_freq) > 0:
            num_cpu_freq = int(CPU_freq)
            if num_cpu_freq > CPU_MAX_FREQ:
                CPU_MAX_FREQ = num_cpu_freq
    print('Max CPU Freq : ' + str(CPU_MAX_FREQ))

CPU freqs: 1586000 1482000 1352000 1248000 1144000 1014000 902000 839000 757000 676000 546000 449000 1768000 1664000 1560000 1352000 1144000 936000
Max CPU Freq : 1768000


* Sometime zygote process can have high cpu usage, when am_proc_start is working.

In [31]:
cpuinfo = lp.log_sections['SERVICE CRITICAL cpuinfo'][:15]

top_cpuinfo = ""
for item in cpuinfo:
    if '%' in item and '/' in item:
        top_cpuinfo = item
        break
        
for item in cpuinfo:
    print(item[:-1])
#display(cpuinfo)

DUMP OF SERVICE CRITICAL cpuinfo:
Load: 11.9 / 12.5 / 13.13
CPU usage from 34054ms to 19987ms ago (2022-04-26 19:52:57.855 to 2022-04-26 19:53:11.922):
  29% 18099/com.facebook.katana: 18% user + 11% kernel / faults: 11565 minor 358 major
  27% 19639/system_server: 17% user + 9.6% kernel / faults: 14697 minor 4184 major
  10% 17118/com.samsung.android.voc: 7.9% user + 2.4% kernel / faults: 16542 minor 164 major
  8.7% 3684/surfaceflinger: 3.9% user + 4.7% kernel / faults: 478 minor 8 major
  8.4% 863/kswapd0: 0% user + 8.4% kernel
  4.5% 2733/com.security.antivirus.clean: 2.8% user + 1.7% kernel / faults: 1496 minor 195 major
  4% 1700/mmc-cmdqd/0: 0% user + 4% kernel
  1.9% 19908/com.android.systemui: 1.2% user + 0.7% kernel / faults: 6108 minor 41 major
  1.7% 3607/android.hardware.graphics.composer@2.1-service: 0.4% user + 1.3% kernel / faults: 54 minor 2 major
  0.9% 1945/decon0: 0% user + 0.9% kernel
  0.9% 30922/com.google.android.gms.persistent: 0.4% user + 0.4% kernel / faults:

* FGS : ForeGroud Service<br>
* CEM : Cache EMpty<br>

In [32]:
lines = lp.log_sections['ACTIVITY MANAGER RUNNING PROCESSES']
proecss_lru_list = grep(lines,['Process LRU list'], 500, show=False)

splited_top_cpuinfo = top_cpuinfo.split(" ")
print(splited_top_cpuinfo)
uid_of_top_cpuinfo = ""
name_of_top_cpuinfo = ""
for item in splited_top_cpuinfo:
    if '/' in item:
        splited = item.split("/")
        uid_of_top_cpuinfo = splited[0]
        name_of_top_cpuinfo = splited[1]
        name_of_top_cpuinfo = name_of_top_cpuinfo[:-1]
        print("uid : [{}]".format(uid_of_top_cpuinfo))
        print("process name : [{}]\n\n".format(name_of_top_cpuinfo))
        break

#pid_of_top_cpuinfo = 13680
#name_of_top_cpuinfo = "com.google.android.webview:sandboxed_process0:org.chromium.content.app.SandboxedProcessService0"

current_process = ""
for process_lru in proecss_lru_list:
    if '#' in process_lru:
        if name_of_top_cpuinfo in current_process:
            if uid_of_top_cpuinfo in current_process:
                print('\033[31m' + 'This process had a matched uid.' + '\033[0m')
                print('\033[31m' + current_process + '\033[0m\n')
            else:
                print(current_process + "\n")
        current_process = process_lru
    else:
        current_process += "\n         " + process_lru

    if len(process_lru) is 0:
        break

['', '', '29%', '18099/com.facebook.katana:', '18%', 'user', '+', '11%', 'kernel', '/', 'faults:', '11565', 'minor', '358', 'major\n']
uid : [18099]
process name : [com.facebook.katana]




In [33]:
lines = lp.log_sections['CPU INFO']
CPU_freqs = grep(lines,['Threads'], 25)

--- Nothing ---


# <span style="color:blue">[PACKAGES]</span> 

* Installed packages and version.

In [34]:
lines = lp.log_sections['SERVICE package']
package_lines = grep(lines,['Package \[', 'versionName', 'versionCode', 'firstInstallTime', 'lastUpdateTime'], 1, show=False)

for package_line in package_lines:
    if 'Package' in package_line:
        print('')
        
    print(package_line)

--- Nothing ---
